In [ ]:
import asyncio
import logging
from azure.ai.tools.synthetic.simulator import (
    OpenAIChatCompletionsModel,
    KeyVaultAPITokenManager,
    Simulator,
    Persona,
)

# Define token manager to get Azure Open AI key from key-vault

In [ ]:
token_manager = KeyVaultAPITokenManager(
    secret_identifier="https://your-keyvault.vault.azure.net/secrets/aoai-secret-name/some-id-here",
    auth_header="api-key",
    logger=logging.getLogger("assistant_bot_token_manager"),
)

# Define system connection and user connection for large language models (LLM)

The Azure AI Simulator can simulate interactions between your large language model (i.e. "agent bot") and a system large language model (i.e. "human bot"). First we start by defining the connection to these two large language models to simulate an interaction. In this example we are simulating a conversation. For the human bot simulating the conversation with your agent bot, we recommend using Azure OpenAI GPT-4 model for the best outcomes. Learn more about [Azure Open AI REST APIs.](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning)

In [ ]:
# Endpoint can be found in your Azure OpenAI studio portal -> completions -> view code
model_name = "model-name"
aoai_resource_name = "aoai-resource-name"
deployment_name = "deployment-name"
endpoint_url = f"https://{aoai_resource_name}.openai.azure.com/openai/deployments/{deployment_name}/chat/completions"

# For simplicity, we define both the system bot and agent bot using the same LLM model.

system_bot_model = OpenAIChatCompletionsModel(
    endpoint_url=endpoint_url,
    token_manager=token_manager,
    api_version="2023-03-15-preview",
    name=model_name,
    max_tokens=300,
    temperature=0.0,
)

agent_bot_model = OpenAIChatCompletionsModel(
    endpoint_url=endpoint_url,
    token_manager=token_manager,
    api_version="2023-03-15-preview",
    name=model_name,
    max_tokens=300,
    temperature=0.0,
)

# Create a persona and assign a task to the human bot

The human bot can be prescribed a persona such as a name, profile and tone (all required parameters) that it will use to simulate the conversation with your agent bot. Provide a task to the human bot that will guide it in simulating the proper responses to your agent bot. The number of tasks will define the number of complete conversations that the simulator will simulate. The metadata (optional parameter which can either be a dictionary of key value pairs or a string text) will provide context to your agent bot on the human bot to answer appropriately.

In [ ]:
# Initialize persona
customPersona = Persona(
    name="Jane",
    profile="Jane Doe is a 28-year-old outdoor enthusiast who lives in Seattle, Washington. She has a passion for exploring nature and loves going on camping and hiking trips with her friends. She has recently become a member of the company's loyalty program and has achieved Bronze level status."
    "Jane has a busy schedule, but she always makes time for her outdoor adventures. She is constantly looking for high-quality gear that can help her make the most of her trips and ensure she has a comfortable experience in the outdoors."
    "Recently, Jane purchased a TrailMaster X4 Tent from the company. This tent is perfect for her needs, as it is both durable and spacious, allowing her to enjoy her camping trips with ease. The price of the tent was $250, and it has already proved to be a great investment."
    "In addition to the tent, Jane also bought a Pathfinder Pro-1 Adventure Compass for $39.99. This compass has helped her navigate challenging trails with confidence, ensuring that she never loses her way during her adventures."
    "Finally, Jane decided to upgrade her sleeping gear by purchasing a CozyNights Sleeping Bag for $100. This sleeping bag has made her camping nights even more enjoyable, as it provides her with the warmth and comfort she needs after a long day of hiking.",
    # A default tone can be "neutral"
    tone="happy",
    meta_data=dict(
        customer_info="## customer_info      name: Jane Doe    age: 28     phone_number: 555-987-6543     email: jane.doe@example.com     address: 789 Broadway St, Seattle, WA 98101      loyalty_program: True     loyalty_program Level: Bronze        ## recent_purchases      order_number: 5  date: 2023-05-01  item: - description:  TrailMaster X4 Tent, quantity 1, price $250    item_number: 1   order_number: 18  date: 2023-05-04  item: - description:  Pathfinder Pro-1 Adventure Compass, quantity 1, price $39.99    item_number: 4   order_number: 28  date: 2023-04-15  item: - description:  CozyNights Sleeping Bag, quantity 1, price $100    item_number: 7"
    ),
)
# Define tasks for the persona
task="Jane is trying to accomplish the task of finding out the best hiking backpacks suitable for her weekend camping trips, and how they compare with other options available in the market. She wants to make an informed decision before making a purchase from the outdoor gear company's website or visiting their physical store."\
        "Jane uses Google to search for 'best hiking backpacks for weekend trips,' hoping to find reliable and updated information from official sources or trusted websites. She expects to see a list of top-rated backpacks, their features, capacity, comfort, durability, and prices. She is also interested in customer reviews to understand the pros and cons of each backpack."\
        "Furthermore, Jane wants to see the specifications, materials used, waterproof capabilities, and available colors for each backpack. She also wants to compare the chosen backpacks with other popular brands like Osprey, Deuter, or Gregory. Jane plans to spend about 20 minutes on this task and shortlist two or three options that suit her requirements and budget."\
        "Finally, as a Bronze level member of the outdoor gear company's loyalty program, Jane might also want to contact customer service to inquire about any special deals or discounts available on her shortlisted backpacks, ensuring she gets the best value for her purchase."


# Initialize the simulator with system and agent bot

In [ ]:
simulator = Simulator(
    systemConnection = system_bot_model,
    userConnection = agent_bot_model
)

# Using asynchronous API to generate a conversation

In [ ]:
conv = await simulator.simulate_async(customPersona, task, 4)

for c in conv["conversation"]:
    print(f"{c['turn_number']}")
    print(f"{c['response']}")
    print("-------------------")

# Using synchronous API to generate a conversation

In [ ]:
conv = simulator.simulate(customPersona, task, 4)

for c in conv["conversation"]:
    print(f"{c['turn_number']}")
    print(f"{c['response']}")
    print("-------------------")

# Initialize simulator with callback

A callback function can be defined which is called when it is the agent bot's turn to generate a response. Conversation history and metadata are included as parameters.

In [ ]:
def service_call(question, conversation_history, meta_data):
    return f"The question was {question}."

async def sim_callback(question, conversation_history, meta_data):
    # you may also await async call
    return service_call(question, conversation_history, meta_data)

In [ ]:
simulator = Simulator(
    simulate_callback = sim_callback,
    systemConnection = system_bot_model
)

# Save and load persona configurations
Once you've defined a custom persona you can export and save it as a YAML file to be imported later.

In [ ]:
persona_export_path = "./custom_persona.yaml"
customPersona.export(persona_export_path)

imported_persona = Persona.load(persona_export_path)
print(imported_persona.__dict__)